<a href="https://colab.research.google.com/github/rajathpatel23/Chord-implementation/blob/master/Resnet_finetune_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import keras
from keras.applications.resnet import ResNet50

In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model, load_model
from pickle import dump
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.regularizers import l2
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
import os
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(False)
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
try:
 device_name = os.environ['COLAB_TPU_ADDR']
 TPU_ADDRESS = 'grpc://' + device_name
 print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
 print('TPU not found')

Found GPU at: /device:GPU:0
TPU not found


In [5]:
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
train_dict_1 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_1.pkl')

In [0]:
train_dict_2 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_2.pkl')

In [0]:
def get_tuning(X_in):
      tuned_out = []
      for x in range(len(X_in)):
            if X_in[x][0] >= 0.5:
                  tuned_out.append(1)
            else:
                  tuned_out.append(0)
      return tuned_out

def get_confusion_matrix(y_t, y_pred):
      tn, fp, fn, tp = confusion_matrix(y_t, y_pred).ravel()
      sensitivity = tp/(tp + fn)
      specificity = tn/(tn + fp)
      PPV = tp/(tp + fp)
      NPV = tn/(tn + fn)
      return sensitivity, specificity, PPV, NPV

def get_aucpr(y_t, y_pred):
      precision, recall, thres  = precision_recall_curve(y_t, y_pred)
      aucpr = auc(recall, precision)
      return aucpr


def get_processed_X(X):
      X_copy  = X
      for arr in range(len(X_copy)):
            a = np.min(X_copy[arr])
            b = np.max(X_copy[arr])
            X_copy[arr] = np.divide(np.subtract(X_copy[arr], a), (b-a))
      return X_copy 
#%%
def get_reshape(X):
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
      return X

In [0]:
X = np.concatenate((train_dict_1["stft"], train_dict_2["stft"]))
y = np.concatenate((train_dict_1["MT"], train_dict_2["MT"]))

In [10]:
# del train_dict_1
# del train_dict_2
X_new = [0 for x in range(len(X))]
dim = (224, 224)
i = 0
for elem in X:
  elem_new = cv2.resize(elem, dim, interpolation = cv2.INTER_AREA)
#   print(elem_new.shape)
  X_new[i] = (elem_new)  
  i+=1

del X
X_new = np.array(X_new)
X_new = get_processed_X(X_new)
X_new = np.stack((X_new,)*3, axis=-1)
print(X_new.shape)

(7683, 224, 224, 3)


In [0]:
# X = np.stack((X,)*3, axis=-1)
# print(X.shape)

In [11]:
model_1 = ResNet50(include_top=True, input_shape=(224, 224, 3))
# print(model_1.summary())
model_1.layers.pop()
# model_1.summary()








102973440/102967424 [==============================] - 4s 0us/step


In [0]:
for layer in model_1.layers:
  layer.trainable = False

In [14]:
# average_layer =GlobalAveragePooling2D()(model_1.output)
# # flat1 =Flatten()(average_layer)
# class1 = Dense(1024, activation='relu')(average_layer)
# class2 = Dense(512, activation='relu')(class1)
# output = Dense(1, activation='sigmoid', activity_regularizer=l2(0.01))(class2)
output = Dense(1, activation='sigmoid', activity_regularizer=l2(0.01))(model_1.layers[-1].output)
model = Model(inputs = model_1.inputs, outputs = output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [15]:
model.compile(loss='binary_crossentropy',optimizer='adam')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
from sklearn.utils import class_weight
from keras.callbacks import ModelCheckpoint, EarlyStopping

sample_weights = class_weight.compute_sample_weight(class_weight="balanced", y=y)
callbacks = [EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True), 
             ModelCheckpoint(filepath='/content/drive/My Drive/Dataset/Resnet_model_new/Resnet_freeze_09172019.h5', monitor='loss', save_best_only=True)]
history = model.fit(X_new, y,epochs=40,batch_size=64,shuffle=True, verbose=1, 
                    sample_weight=sample_weights, callbacks=callbacks )

Epoch 1/40
7683/7683 [==============================] - 65s 8ms/step - loss: 0.7491
Epoch 2/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.6171
Epoch 3/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.5939
Epoch 4/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.5420
Epoch 5/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.5263
Epoch 6/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.5144
Epoch 7/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.4935
Epoch 8/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.4807
Epoch 9/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.4693
Epoch 10/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.4693
Epoch 11/40
7683/7683 [==============================] - 60s 8ms/step - loss: 0.4322
Epoch 12/40
7683/7683 [==============================] - 60s 8ms/step - lo

In [5]:
model = load_model('/content/drive/My Drive/Dataset/Resnet_freeze_09172019.h5')
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, auc, roc_curve
from sklearn.metrics import average_precision_score, confusion_matrix, precision_recall_curve, classification_report
def performance_analysis(y_2, y_test_1):      
      tuned_y_2 = get_tuning(y_2)
      roc_auc_test = roc_auc_score(y_test_1, y_2)
      print("The test roc is : ", roc_auc_test)
      aucpr_test = get_aucpr(y_test_1, y_2)
      print("The test aucpr is: ", aucpr_test)
      sense_test_, specs_test_, PPV_test_, NPV_test_ = get_confusion_matrix(y_test_1, tuned_y_2)
      print("The test sensitivity is: ", sense_test_)
      print("The test specificity is: ", specs_test_)
      print("The test PPV is: ", PPV_test_)
      print("The test NPV is: ", NPV_test_)
      return True









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
from sklearn.externals import joblib
test_1 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_1.pkl')
test_2 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_2.pkl')

In [0]:
X = np.concatenate((test_1["stft"], test_2["stft"]))
y_test = np.concatenate((test_1["MT"], test_2["MT"]))
del test_1
del test_2

In [30]:
X_new = [0 for x in range(len(X))]
dim = (224, 224)
i = 0
for elem in X:
  elem_new = cv2.resize(elem, dim, interpolation = cv2.INTER_AREA)
#   print(elem_new.shape)
  X_new[i] = (elem_new)
  i+=1

del X
X_new = np.array(X_new)
X_new = get_processed_X(X_new)
X_new = np.stack((X_new,)*3, axis=-1)
print(X_new.shape)

(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)

In [31]:
y_pred_MT = model.predict(X_new)
performance_analysis(y_pred_MT, y_test)

The test roc is :  0.2814767537790943
The test aucpr is:  0.01267656067735149
The test sensitivity is:  0.0
The test specificity is:  1.0
The test PPV is:  nan
The test NPV is:  0.9797979797979798


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in long_scalars
  


True

In [0]:
model.save('/content/drive/My Drive/Dataset/Resnet50_1_40_64_MT_STFT.h5')